In [2]:
from pathlib import Path
from multiprocessing import Pool
from functools import partial

import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image

import matplotlib.pyplot as plt
# %matplotlib inline

In [14]:
DATASET_PATH = Path("/media") / "storage" / "COVIDxCT"
SPLIT = "test"

df = pd.read_csv(DATASET_PATH / f"{SPLIT}_COVIDx_CT-2A.txt",
                 delimiter=" ",
                 names=["filename", "class", "xmin", "ymin",
                        "xmax", "ymax"])
# train_df = train_df.sample(frac=1., random_state=1000).reset_index(drop=True)
filenames = df["filename"].to_numpy()
filepaths = [DATASET_PATH / "2A_images" / filename for filename in filenames]
print(len(filenames))
df.head()

25658


,filename,class,xmin,ymin,xmax,ymax
0,NCP_341_1834_0022.png,2,0,130,512,429
1,NCP_341_1834_0023.png,2,0,130,512,429
2,NCP_341_1834_0024.png,2,0,130,512,429
3,NCP_341_1834_0029.png,2,0,128,512,430
4,NCP_341_1834_0030.png,2,0,128,512,430


In [15]:
def convert_to_jp2(filepath: str, output_dir: str):
    filepath = Path(filepath)
    filename = filepath.stem
    img = Image.open(filepath)

    filepath = Path(output_dir) / f"{filename}.jp2"
    img.save(str(filepath))


# img_dir = DATASET_PATH / "2A_images"
img_dir = "/home/hoangtnm/Downloads/test"

with Pool(processes=4) as pool:
    with tqdm(total=len(filenames)) as pbar:
        func = partial(convert_to_jp2, output_dir=img_dir)
        for i, _ in enumerate(pool.imap_unordered(func, filepaths)):
            pbar.update()

100%|██████████| 25658/25658 [06:36<00:00, 64.78it/s] 


In [ ]:
for i in range(1):
    data = df.iloc[0]
    filepath = Path(DATASET_PATH) / "2A_images" / data["filename"]
    f = open(filepath, "rb")
    xmin, ymin = data["xmin"], data["ymin"]
    xmax, ymax = data["xmax"], data["ymax"]

    img = Image.open(filepath).convert("RGB")
    # img = img.crop((xmin, ymin, xmax, ymax))
    img_np = np.array(img)
    
    img.save("test.jp2")
    img_jp2 = Image.open("test.jp2")
    img_jp2_np = np.array(img_jp2)
    
    if not np.all(img_np==img_jp2_np):
        print(i)

In [ ]:
plt.imshow(img)
# plt.imshow(img_np)